In [1]:
import os
DEVELOPMENT_MODE = False
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2
        
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
# if IN_COLAB or not DEVELOPMENT_MODE:
#     pio.renderers.default = "colab"
# else:
#     pio.renderers.default = "notebook_connected"
pio.renderers.default = 'notebook_connected' #1回設定しておけばいい！
print(f"Using renderer: {pio.renderers.default}")

import circuitsvis as cv
# Testing that the library works
cv.examples.hello("keno")

# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial

# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

Using renderer: notebook_connected


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "microsoft/Phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
# tf_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
model = HookedTransformer.from_pretrained_no_processing(model_id, torch_dtype=torch.bfloat16, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


In [3]:
tokenizer.encode(':')

[128000, 25]

In [4]:
clean_prompt = """Write a high-quality answer for the given question using only the provided search results (some of which might be irrelevant).

Document [1](Title: Nobel Prize in Physics) receive a diploma, a medal and a document confirming the prize amount. Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was
Document [2](Title: Norwegian Americans) science, Ernest Lawrence won the Nobel Prize in Physics in 1939. Lars Onsager won the 1968 Nobel Prize in Chemistry. Norman Borlaug, father of the Green Revolution, won the Nobel Peace Prize in 1970. Christian B. Anfinsen won the Nobel Prize for chemistry in 1972. Ivar Giaever won the Nobel Prize in Physics 1973. Carl Richard Hagen is noted for his work in physics. In engineering, Clayton Jacobson II is credited with the invention of the modern personal watercraft. Ole Singstad was a pioneer of underwater tunnels. Ole Evinrude invented the first outboard motor with practical commercial application, recognizable today
Document [3](Title: Nobel Prize in Physics) Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was awarded to physicist Wilhelm Röntgen in recognition of the extraordinary services he
Document [4](Title: École normale supérieure (Paris)) was also awarded the Abel prize. In addition, eight "normaliens" have gone on to receive the Nobel Prize in Physics: Claude Cohen-Tannoudji, Pierre-Gilles de Gennes, Albert Fert, Alfred Kastler, Gabriel Lippmann, Louis Néel, Jean Baptiste Perrin and Serge Haroche, while other ENS physicists include such major figures as Paul Langevin, famous for developing Langevin dynamics and the Langevin equation. Alumnus Paul Sabatier won the Nobel Prize in Chemistry. A ranking of universities worldwide based on ratios of alumni to Nobel prize-winners published in 2016 by American scholars Stephen Hsu and Jonathan Wai placed ENS as the first university worldwide, far
Document [5](Title: List of Nobel laureates in Physics) The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen, of Germany, who received 150,782 SEK, which is equal to 7,731,004 SEK in December 2007.  John Bardeen is the only laureate to win the prize twice—in 1956 and 1972. Maria Skłodowska-Curie also won two Nobel Prizes, for physics in 1903 and chemistry in 1911. William Lawrence Bragg was, until October 2014, the youngest ever Nobel laureate; he won the prize in 1915 at the age of 25. Two women have won the prize: Curie and Maria Goeppert-Mayer (1963). As of 2017, the prize has been awarded
Document [6](Title: Nobel Prize in Physics) rendered by the discovery of the remarkable rays (or x-rays). This award is administered by the Nobel Foundation and widely regarded as the most prestigious award that a scientist can receive in physics. It is presented in Stockholm at an annual ceremony on 10 December, the anniversary of Nobel's death. Through 2018, a total of 209 individuals have been awarded the prize. Only three women (1.4% of laureates) have won the Nobel Prize in Physics: Marie Curie in 1903, Maria Goeppert Mayer in 1963, and Donna Strickland in 2018. Alfred Nobel, in his last will and testament, stated that his
Document [7](Title: E. C. George Sudarshan) had developed the breakthrough. In 2007, Sudarshan told the "Hindustan Times", "The 2005 Nobel prize for Physics was awarded for my work, but I wasn't the one to get it. Each one of the discoveries that this Nobel was given for work based on my research." Sudarshan also commented on not being selected for the 1979 Nobel, "Steven Weinberg, Sheldon Glashow and Abdus Salam built on work I had done as a 26-year-old student. If you give a prize for a building, shouldn’t the fellow who built the first floor be given the prize before those who built the second
Document [8](Title: Svante Arrhenius) Wilhelm Ostwald, Theodore Richards) and to attempt to deny them to his enemies (Paul Ehrlich, Walther Nernst, Dmitri Mendeleev). In 1901 Arrhenius was elected to the Swedish Academy of Sciences, against strong opposition. In 1903 he became the first Swede to be awarded the Nobel Prize in chemistry. In 1905, upon the founding of the Nobel Institute for Physical Research at Stockholm, he was appointed rector of the institute, the position where he remained until retirement in 1927. He was elected a Foreign Member of the Royal Society (ForMemRS) in 1910. In 1911 he won the first Willard Gibbs Award.
Document [9](Title: Scientist) and pursued through a unique method, was essentially in place. Ramón y Cajal won the Nobel Prize in 1906 for his remarkable observations in neuroanatomy. Marie Curie became the first female to win the Nobel Prize and the first person to win it twice. Her efforts led to the development of nuclear energy and Radio therapy for the treatment of cancer. In 1922, she was appointed a member of the International Commission on Intellectual Co-operation by the Council of the League of Nations. She campaigned for scientist's right to patent their discoveries and inventions. She also campaigned for free access
Document [10](Title: Nobel Prize in Chemistry) on December 10, the anniversary of Nobel's death. The first Nobel Prize in Chemistry was awarded in 1901 to Jacobus Henricus van 't Hoff, of the Netherlands, "for his discovery of the laws of chemical dynamics and osmotic pressure in solutions." From 1901 to 2018, the award has been bestowed on a total of 180 individuals. Alfred Nobel stipulated in his last will and testament that his money be used to create a series of prizes for those who confer the "greatest benefit on mankind" in physics, chemistry, peace, physiology or medicine, and literature. Though Nobel wrote several wills during

Question: who got the first nobel prize in physics
Answer: Wilhelm Conrad Röntgen

Document [1](Title: Deadpool (video game)) 2015 in North America and Australia, and November 20, 2015 in the United Kingdom. It is unknown how Activision reacquired the rights to publish "Deadpool: The Video Game", but it is believed that the re-release was timed to help promote the then-upcoming Deadpool feature film, released roughly three months after the next-gen version of the game. High Moon Studios showed an early demo of the video game to journalists at Gamescom 2012, where the game earned a nomination for Best of Show. Several sites wrote positive impression of the demo that was shown at Gamescom and the Electronic Entertainment Expo
Document [2](Title: Deadpool (film)) footage was leaked online in July 2014, and was met with an overwhelmingly enthusiastic response. That September, Fox gave "Deadpool" a release date of February 12, 2016. Production was set to begin in March 2015, with Simon Kinberg joining as producer. Reynolds attributed Fox's green-lighting of the film entirely to the leak. He, Miller and the writers had previously discussed leaking the footage themselves, and Reynolds initially thought that Miller had done so. He later believed the leak came from someone at Fox. In exchange for being able to make the film the way they wanted, Fox gave the crew
Document [3](Title: Deadpool (film)) May of that year. Several vendors provided visual effects for the film, ranging from the addition of blood and gore to the creation of the CG character Colossus. "Deadpool" was released in the United States on February 12, 2016, after an unconventional marketing campaign. The film achieved both financial and critical success. It earned over $783 million against a $58 million budget, breaking numerous records: it became the highest-grossing R-rated film, the highest-grossing "X-Men" film, and the ninth-highest-grossing 2016 film. Critics praised Reynolds' performance, the film's style and faithfulness to the comics, and its action sequences. Some detractors criticized the
Document [4](Title: Paul Wernick) Tim Miller. The R-rated film was released on February 12, 2016. Its opening weekend was the highest grossing of all time for R-Rated films, and it went on to gross more than $780 million worldwide. The team's next project, sci-fi thriller "Life", was released in March 2017. The film was directed by Daniel Espinosa, produced by Skydance, and through Sony Pictures, with Ryan Reynolds, Jake Gyllenhaal, and Rebecca Ferguson starring. They wrote "Deadpool 2" in 2018, which like its predecessor was released to positive reviews. Wernick and Reese were announced as the co-screenwriters of the feature film adaptation of "Cowboy
Document [5](Title: Deadpool 2) Deadpool 2 is scheduled to be released in the United States on May 18, 2018.  A sequel, Deadpool 3, is in development.
Document [6](Title: Deadpool 2) film, explaining its general plot, was released at the end of March. "Forbes" contributor Scott Mendelson called it "pretty funny and mostly entertaining", but was disappointed in it being a "conventional" trailer compared to the more out-there videos previously released for the film. He explained that he thought the first film "had a winning lead character and fine character-centric jokes, but a pretty generic origin story plot that eventually became the thing it was critiquing", and was concerned that the sequel would turn out to be "a more standard 'superhero sequel' sell". Mendelson also noted the inclusion of T.J. Miller
Document [7](Title: Deadpool 2) at the convention moderated by Soni and featuring other cast members. Some of the scenes cut from the theatrical version of the film were debuted at the panel. At the end of September 2018, Fox announced that it would release an untitled "Deadpool" film in theaters on December 21. The studio suggested that press and fans "guess away" as to what this new film would be, but it was believed to be a re-cut version of "Deadpool 2" that would carry a PG-13 rating rather than being R-rated like the initial theatrical release. Reynolds hinted that the version of the
Document [8](Title: The New Mutants (film)) Bear would not be the main antagonist of the film, just that it was "very much inspired" by the run of the "New Mutants" comics in which Demon Bear was featured as the main villain. In January 2018, the film's release date was pushed back to February 22, 2019, which allowed it to avoid "Deadpool 2", which also had its release changed at that time to a date that would have had both films in theaters at the same time in certain markets, while also allowing time for the reshoots required to make the film more frightening. When asked about
Document [9](Title: Deadpool (film)) said that the company would be willing to make future R-rated Marvel films like "Deadpool", potentially under a "Marvel-R" brand, "as long as we let the audiences know what's coming". Before "Deadpool"s release, Fox green-lit a sequel with Reese and Wernick returning to write the screenplay. The involvement of Reynolds and Tim Miller was confirmed at the 2016 CinemaCon in April, but at the end of October, Miller left the film over "mutual creative differences" with Reynolds. The next month, David Leitch signed on to replace Miller for the sequel. Leitch first made a short film, "No Good Deed", which
Document [10](Title: Deadpool (film)) Screen Rant called it possibly "the best film marketing campaign in the history of cinema". HostGator's Jeremy Jensen attributed the campaign's success to Reynolds, and to Fox for embracing the film's R rating. "Deadpool"s world premiere was held at the Grand Rex in Paris on February 8, 2016, before its initial theatrical release in Hong Kong the next day. This was followed by releases in 49 other markets over the next few days, including the United States on February 12. The movie was released in several formats, including IMAX, DLP, premium large formats, and D-Box. Kinberg explained that unlike the

Question: when is the next deadpool movie being released
Answer: May 18, 2018

Document [1](Title: Geography of Nigeria) south atlantic ocean, locally known as the south western wind, or by its main name, The Tropical Maritime (MT) airmass. These two major wind systems in Nigeria are known as the trade winds. The tropical maritime airmass (MT) is responsible for Nigeria's rainy season. This wind (the tropical maritime airmass) invades the country from February in the southern part of Nigeria while it takes longer for the wind to fully cover the whole of the country, reaching the northern part of Nigeria in June. Its invasion is as a result of the northward retreat, of the tropical continental airmass (CT)
Document [2](Title: Geography of Nigeria) of low pressure to develop over west Africa and Nigeria (between March to May). The Tropical continental airmass (CT) from the Sahara Desert in the northern part of West Africa, is weakened due to the overheating of the land surface in west Africa and Nigeria at this time. The Tropical continental airmass (CT) begins to retreat northwards to the Sahara Desert due to massive heating of the land which transfers heat in the form of convection into the Tropical continental airmass (CT) which constitutes the main layer of air above the land. This transfer of heat in the Tropical continental
Document [3](Title: Geography of Nigeria) airmass (CT) in turn, causes the wind to expand and become lighter as this is the normal behaviour for winds moving above intensely heated grounds. The Tropical continental airmass (CT) loses its strength as a major airmass in the region of west Africa and over Nigeria at this time (around February in the southern part of Nigeria to June in northern Nigeria) and begins to retreat coupled with the rising of air in form of convection within this airmass (Tropical continental airmass (CT)), further weakening the dominance of the wind over west Africa and Nigeria. The Tropical continental airmass (CT)
Document [4](Title: Geography of Nigeria) northern end is south of the 15 degrees line at about 14 degrees. Nigeria's location in the wetter part of the easterly waves south of the 15 degree line creates wetter climatic conditions for Nigeria especially during the monsoons. The Tropical Continental Airmass (CT) locally known as the harmattan, is a wind originating from North Africa which crosses the Sahara Desert into west Africa to Nigeria. This airmass dominates Nigeria's climate during the dry season from December to March. The Tropical continental airmass is dusty and creates a haze within the atmosphere of west Africa and Nigeria when it predominates.
Document [5](Title: Geography of Nigeria) With the Intertropical Convergence Zone (ITCZ) swinging northward over West Africa from the Southern Hemisphere in April, heavy showers coming from pre-monsoonal convective clouds mainly in the form of squall lines also known as the north easterlies formed mainly as a result of the interactions of the two dominant airmasses in Nigeria known as the Maritime tropical(south westerlies) and the Continental tropical(north easterlies), begins in central Nigeria while the Monsoons from the south atlantic ocean arrives in central Nigeria in July bringing with it high humidity, heavy cloud cover and heavy rainfall which can be daily occurrence lasting till September when the monsoons gradually begin retreating southward to the southern part of Nigeria.
Document [6](Title: Awka) dry winds from across the Sahara desert. The monsoon winds from the Atlantic creates six months of heavy tropical rains, which occur between April and July, followed by a short dry period in August lasting two to three weeks with the rain resuming in September and October. This is followed by five months of dryness (November - March) marked by a Harmattan wind, also known as Ugulu in Igbo, which is a particularly dry and dusty wind which enters Nigeria in late December or in the early part of January and is characterized by a grey haze limiting visibility and
Document [7](Title: Geography of Nigeria) experiences inward blowing winds because winds are moving air blowing outwards from regions of high pressure to regions of low pressure. The Tropical Maritime Airmass is a warm humid and unstable trade wind due to its warmth. Convectional currents are easily set up within the airmass whenever there is little instability in the airmass as a result of a slight to a very high orographic uplift in mountainous regions like the obudu plateau or the heating of the land which can trigger the formation of cumulonimbus cloud leading to thunderstorms within the airmass. During the dominance of the Tropical Maritime
Document [8](Title: Geography of Nigeria) finally retreats from most part of Nigeria, and the West African atmosphere around April to May, leaving an empty atmosphere over Nigeria. The sun's rays enters into the atmosphere of Nigeria more intense than it does during the presence of the Tropical continental airmass, which contained dust (in form of haze) that reduced the intensity of the sun. The overheating of the west Africa land mass and Nigeria in particular creates a low pressure region over west Africa and Nigeria. This low pressure zone attracts the Tropical Maritime Airmass (MT) from the south Atlantic Ocean since areas of low pressures
Document [9](Title: Western Asia) two wind phenomena in Western Asia: the "sharqi" and the "shamal". The "sharqi" (or "sharki") is a wind that comes from the south and southeast. It is seasonal, lasting from April to early June, and comes again between late September and November. The winds are dry and dusty, with occasional gusts up to 80 kilometres per hour (50 miles per hour) and often kick up violent sand and dust storms that can carry sand a few thousand meters high, and can close down airports for short periods of time. These winds can last for a full day at the beginning
Document [10](Title: Onikwu) The dry season is accompanied by a dust laden airmass from the Sahara Desert, locally known as Harmattan, or by its main name, The Tropical Continental (CT) airmass, while the rainy season is heavily influenced by an airmass originating from the south Atlantic Ocean, locally known as the south west wind, or by its main name, The Tropical Maritime (MT) airmass. These two major wind systems in Nigeria are known as the trade winds. The region Onikwu/Ndoni is flood prone communities, this is because the inland part of Rivers state consists of tropical rainforest; towards the coast the typical Niger

Question: the south west wind blows across nigeria between
Answer:"""

clean_tokens = tokenizer(clean_prompt, return_tensors="pt").input_ids.to('cuda')
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
tokenizer.decode(clean_logits[0, -1, :].argmax())

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.18 GiB. GPU 

In [ ]:
len(clean_tokens[0])

4279